# TSMixer

In [ ]:
!pip install pytorch-tsmixer

In [42]:
import torch

def smape_loss(predicted, target):
    """
    Compute Symmetric Mean Absolute Percentage Error (SMAPE) loss between predicted and target tensors.
    Args:
        predicted (torch.Tensor): Predicted values.
        target (torch.Tensor): Target values.
    Returns:
        torch.Tensor: SMAPE loss.
    """
    numerator = torch.abs(predicted - target)
    denominator = (torch.abs(predicted) + torch.abs(target)) / 2.0
    smape = torch.mean(numerator / denominator) * 100.0
    return smape

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtsmixer import TSMixer
import pandas as pd 
import sys
from torch.utils.data import DataLoader, TensorDataset

# Consider sentiment score? 
dropScore = True


ts = pd.read_csv('covidDataWithSentiment.csv')

# Drop cumulative datas?
cumulative_clmsn = []
for clmn_name in ts:
    if "Cumulative" in clmn_name:
        cumulative_clmsn.append(clmn_name)
    
    if dropScore == True and "score" in clmn_name:
        cumulative_clmsn.append(clmn_name)
        
for entry in cumulative_clmsn:
    ts.drop(columns=entry, inplace=True)

# Drop date
data = ts.select_dtypes(include=['float64', 'float32', 'float16', 'int64', 'int32', 'int16', 'int8', 'uint8'])
data = data.fillna(0)

# Model parameters
sequence_length = 14
prediction_length = 5
input_channels = len(data.columns) - 1
output_channels = 1


X_train = torch.empty(0, sequence_length, input_channels)
y_train = torch.empty(0, prediction_length, output_channels)

for start in range(1, len(data) - sequence_length - prediction_length + 1):
    train_inst = torch.tensor(data.iloc[start:start + sequence_length, :-1].values, dtype=torch.float32)
    label_inst = torch.tensor(data.iloc[start + sequence_length : start + sequence_length + prediction_length, -1:].values, dtype=torch.float32)    

    X_train = torch.cat((X_train, train_inst.unsqueeze(0)), dim = 0)
    y_train = torch.cat((y_train, label_inst.unsqueeze(0)), dim = 0)

print(X_train.shape)
print(y_train.shape)


dataset = TensorDataset(X_train, y_train)
# Create a DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)



# Create the TSMixer model
model = TSMixer(sequence_length, prediction_length, input_channels, output_channels)

# Loss function and optimizer
criterion = smape_loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 1500
for epoch in range(num_epochs):
    model.train()
    for X, y in dataloader:
        # Extract a single batch from the dataset   
                
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X) 
        loss = criterion(outputs, y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete")

torch.Size([401, 14, 12])
torch.Size([401, 5, 1])
Epoch [1/1500], Loss: 199.9692
Epoch [2/1500], Loss: 199.9690
Epoch [3/1500], Loss: 199.9668
Epoch [4/1500], Loss: 199.9684
Epoch [5/1500], Loss: 199.9701
Epoch [6/1500], Loss: 199.9624
Epoch [7/1500], Loss: 199.9681
Epoch [8/1500], Loss: 199.9723
Epoch [9/1500], Loss: 199.9670
Epoch [10/1500], Loss: 199.9575
Epoch [11/1500], Loss: 199.9661
Epoch [12/1500], Loss: 199.9644
Epoch [13/1500], Loss: 199.9696
Epoch [14/1500], Loss: 199.9677
Epoch [15/1500], Loss: 199.9672
Epoch [16/1500], Loss: 199.9708
Epoch [17/1500], Loss: 199.9635
Epoch [18/1500], Loss: 199.9679
Epoch [19/1500], Loss: 199.9674
Epoch [20/1500], Loss: 199.9636
Epoch [21/1500], Loss: 199.9632
Epoch [22/1500], Loss: 199.9696
Epoch [23/1500], Loss: 199.9610
Epoch [24/1500], Loss: 199.9647
Epoch [25/1500], Loss: 199.9646
Epoch [26/1500], Loss: 199.9590
Epoch [27/1500], Loss: 199.9595
Epoch [28/1500], Loss: 199.9598
Epoch [29/1500], Loss: 199.9570
Epoch [30/1500], Loss: 199.9582